# Fuzzy-Lookup

In [23]:
# We import the libraries that we are goin to use. 

import numpy as np
import pandas as pd
from polyfuzz.models import TFIDF

## Using the Account Name parameter

### Cleansing the SF accounts

In [24]:
# We read the file that contains the SF Accounts.

accounts_path = 'input/sf_accounts.csv'

df2 = pd.read_csv(accounts_path, encoding = 'Latin1').dropna(subset=['Account ID'])

In [25]:
df2.head()

,Account Name,Website,Contact Count,Account ID
0,BH Live,www.bhlive.org.uk,2.0,0011300001wDBbT
1,Telesure group services,NaN,1.0,0014O00002IfXnR
2,Fluent Money Ltd,fluentmoney.co.uk,1.0,0014O00002LCOke
3,Applus+ Automotive,www.applusautomotive.com,1.0,0014O00002LDtT3
4,Riverview LLP,riverviewllp.com,1.0,0014O00002LCuWO


In [26]:
# We stablish the strange symbols and patters that we are going to remove. 

a = '[^a-z0-9\s]| inc| plc| llc| ltd| llp| limited| group| corporation| uk'

In [27]:
# We create a new Account Name column to clean it.
# We remove those symbols and patters from the Account Name Cleaned column.
# We replace the empty strings with NaN values in the Account Name Cleaned column.

df2['Account Name Cleaned'] = df2['Account Name']
df2['Account Name Cleaned'] = df2['Account Name Cleaned'].str.lower().str.replace(a, '', regex=True).str.strip()
df2['Account Name Cleaned'] = df2['Account Name Cleaned'].replace('', np.nan)

In [28]:
# We create a DataFrame without NaN values in the Account Name Cleaned column. 
# We sort by Account Name Cleaned and Contact Count.
# We drop duplicates in the Account Name Cleaned column and we keep the first one. 

df2_account = df2.dropna(subset=['Account Name Cleaned'])
df2_account = df2_account.sort_values(by=['Account Name Cleaned', 'Contact Count'], ascending=[True, False])
df2_account = df2_account.drop_duplicates(subset='Account Name Cleaned', keep='first')

In [29]:
# We create a list with the Account Name Cleaned values. 

df2_account_list = df2_account['Account Name Cleaned'].tolist()

### Cleansing the external companies

In [30]:
# We read the file that contains the external accounts. 

ext_data_path = 'input/ext_data.csv'

df3 = pd.read_csv(ext_data_path, encoding="Latin1")


In [31]:
# We create a new Account Name column to clean it.
# We remove those symbols and patters from the Account Name Cleaned column.
# We replace the empty strings with NaN values in the Account Name Cleaned column.

df3['Company Name Cleaned'] = df3['Company Name']
df3['Company Name Cleaned'] = df3['Company Name Cleaned'].str.lower().str.replace(a, '', regex=True).str.strip()
df3['Company Name Cleaned'] = df3['Company Name Cleaned'].fillna(' ')

In [32]:
df3.head()

,Unnamed: 0,First Name,Last Name,Email,Phone,Company Name,Company Name Cleaned
0,0,George,Courtot,gcourtot@trimarkusa.com,(508) 399-2400,TriMark USA,trimark usa
1,1,Michael,Farnham,michael.farnham@ge.com,(513) 243-4006,General Electric,general electric
2,3,Brittany,Rogers,brittany.rogers@shawinc.com,(800) 446-9332,"Shaw Industries Group, Inc.",shaw industries
3,4,Barry,Okun,bokun@wegochem.com,(516) 487-3510 ext. 129,Wego Chemical Group,wego chemical
4,5,Quinlan,Miller,quinlan.miller@jbtc.com,(919) 362-8811 ext. 284,JBT Corporation,jbt


In [33]:
# We create a list with the Account Name Cleaned values and we drop the duplicates. 

df3_company_list = df3.drop_duplicates(subset='Company Name Cleaned', keep='first')['Company Name Cleaned'].tolist()

In [34]:
df3_company_list[:5]

['trimark usa', 'general electric', 'shaw industries', 'wego chemical', 'jbt']

In [35]:
# We use the model 'TF-IDF' from the PolyFuzz library.
# Changing Parameters to be used on large Data
model = TFIDF(n_gram_range=(2,2), clean_string=True, min_similarity=0.95, cosine_method='sparse', top_n=1)

In [36]:
#Saving results of model
# Main part of the Fuzzy search
result = model.match(df3_company_list, df2_account_list)

---

In [37]:
result.head()

,From,To,Similarity
0,trimark usa,trimark usa,1.0
1,general electric,general electric,1.0
2,shaw industries,shaw industries,1.0
3,wego chemical,None,0.0
4,jbt,jbt,1.0


In [38]:
# We create a DataFrame that contains similarity > .95

old_account = result[result['Similarity'] > .30].reset_index(drop=True)
old_account = old_account.rename({'From': 'Company Name Cleaned', 'To':  'Account Name Cleaned'}, axis=1)

In [39]:
# old_account['T/F'] = old_account['Account Name Cleaned'] == old_account['Company Name Cleaned']

old_account['T/F'] = np.where(old_account['Similarity'] >= 0.98, True, False)


---

In [40]:
old_account['string_based'] = old_account['Account Name Cleaned'] == old_account['Company Name Cleaned']
old_account['len'] = old_account['Company Name Cleaned'].apply(lambda x: len(x))

---

In [41]:
old_account = old_account.sort_values(by=['T/F'], ascending=True).reset_index(drop=True)

In [43]:
old_account.to_csv('output/old_account_check.csv', index=False)

In [ ]:
exit()

---

In [44]:
old_account = pd.read_csv('output/old_account_check.csv')

In [45]:
old_account.loc[(old_account['T/F'] != old_account['string_based']) & (old_account['len'] <= 20)].sort_values(by='len')

,Company Name Cleaned,Account Name Cleaned,Similarity,T/F,string_based,len
115,crane co,crane co,1.000,True,False,8.0
126,u s steel,steel,1.000,True,False,9.0
130,a o smith,smith,1.000,True,False,9.0
276,wagner equipment co,wagner equipment,0.986,True,False,19.0
284,baker hughes company,baker hughes a ge company,0.975,True,False,20.0


In [46]:
old_account_T = old_account[old_account['T/F'] == True].reset_index(drop='True')
old_account_F = old_account[old_account['T/F'] == False].reset_index(drop='True')

In [47]:
inner_account_T = pd.merge(df2_account,
                      old_account_T,
                      on = 'Account Name Cleaned',
                      how = 'inner',)
                      #indicator = True).drop(columns = '_merge')

In [48]:
inner_account_T

,Account Name,Website,Contact Count,Account ID,Account Name Cleaned,Company Name Cleaned,Similarity,T/F,string_based,len
0,3D Systems,www.3dsystems.com,49.0,0014O000029352o,3d systems,3d systems,1.0,True,True,10.0
1,84 Lumber Company,www.84lumber.com,26.0,0014O00002935Gu,84 lumber company,84 lumber company,1.0,True,True,17.0
2,Accuride International,www.accuride.com,9.0,0014O0000292z7R,accuride international,accuride international,1.0,True,True,22.0
3,Acme Brick,www.brick.com,27.0,0014O0000292z2l,acme brick,acme brick,1.0,True,True,10.0
4,Advanced Drainage Systems,www.ads-pipe.com,15.0,0014O0000292zYR,advanced drainage systems,advanced drainage systems,1.0,True,True,25.0
...,...,...,...,...,...,...,...,...,...,...
354,"W.W. Grainger, Inc.",NaN,11.0,0014O00002917zN,ww grainger,ww grainger,1.0,True,True,11.0
355,XAL,www.xalusa.com,6.0,0014O00002937Wd,xal,xal,1.0,True,True,3.0
356,Xilinx,www.xilinx.com,134.0,0014O0000293I6I,xilinx,xilinx,1.0,True,True,6.0
357,Xylem,www.xylem.com,203.0,0014O0000293L1i,xylem,xylem,1.0,True,True,5.0


In [49]:
inner_account_T = inner_account_T[['Account ID', 'Account Name', 'Company Name Cleaned', 'Website']]

In [50]:
inner_company_T = pd.merge(inner_account_T,
                      df3,
                      on = 'Company Name Cleaned',
                      how = 'left',)
                      #indicator = True).drop(columns = '_merge')

In [51]:
inner_company_T.head()

,Account ID,Account Name,Company Name Cleaned,Website,Unnamed: 0,First Name,Last Name,Email,Phone,Company Name
0,0014O000029352o,3D Systems,3d systems,www.3dsystems.com,245,Jagtar,Narula,jagtar.narula@3dsystems.com,NaN,3D Systems Corporation
1,0014O00002935Gu,84 Lumber Company,84 lumber company,www.84lumber.com,408,Paul,Lentz,paul.lentz@84lumber.com,(724) 228-8820 ext. 1442,84 Lumber Company
2,0014O0000292z7R,Accuride International,accuride international,www.accuride.com,222,Jeffrey,Dunlap,jdunlap@accuride.com,(562) 903-0200 ext. 1851,Accuride International
3,0014O0000292z2l,Acme Brick,acme brick,www.brick.com,769,Sue,Roundy,sroundy@brick.com,(817) 870-8211,Acme Brick
4,0014O0000292zYR,Advanced Drainage Systems,advanced drainage systems,www.ads-pipe.com,288,Scott,Cottrill,scott.cottrill@ads-pipe.com,(614) 658-0149,Advanced Drainage Systems Inc


In [52]:
# Accounts that we have in our system

inner_company_T.to_csv('output/old_accounts.csv') #Old accounts

---

# Concatenation function

In [53]:
# Getting the list of ID's
for i in range(0, len(inner_company_T['Account ID'].drop_duplicates()), 60):
   concat = ','.join(inner_company_T['Account ID'].drop_duplicates().tolist()[i:i+60])
   print(concat, f" size :{len(concat)}")

0014O000029352o,0014O00002935Gu,0014O0000292z7R,0014O0000292z2l,0014O0000292zYR,0014O00002934Q4,001a000001RZDIG,0014O000025jnWr,0014O00002IdrpY,0014O00002938BV,0014O00002EtcCz,0014O000025jo4d,0014O00002933BZ,0014O0000292yeg,0014O0000293CRt,0014O00002938bH,0014O0000292xRz,0014O00002938EY,0011300001wDBSF,0014O0000292DZg,0014O000029338A,0014O00002IeaKy,0014O00002937yN,001a000001RZDMB,0014O00002IdrxM,0014O00002EuyFn,0014O0000293KKi,0014O00002Euy7o,0014O0000292zu2,0014O000025jo4Q,0014O00002Idrrk,0014O000025kB1a,0014O0000292xRV,0014O000029338D,0011300001uCMLD,0014O0000293KuF,0014O0000292zaA,001a000001RZDUu,0014O00002938FN,0011300001e3X1a,0014O00002LDqd4,0014O0000292y3N,0014O000025jmaF,001a000001RZDUz,0014O00002937ZQ,0014O00002LBgw6,0014O000025jqhS,0014O0000293L84,001a000001U9xuB,0014O00002938dy,0014O00002LCEx0,0014O00002938E9,0014O000029352x,0014O000025jme5,0014O00002Euzqt,001a000001RZDa9,001a000001TfuYj,0014O0000292yqE,0014O0000293KrB,001a000001Tfgr3  size :959
0014O00002927tc,0014O00002Idr

---

In [54]:
inner_company_T_list = inner_company_T['Company Name'].tolist()

In [55]:
new_accounts = df3[~df3['Company Name'].isin(inner_company_T_list)].reset_index(drop=True)

In [63]:
new_accounts.to_csv('output/new_accounts.csv', encoding='utf-8')